# Workshop Rubin Kickstarter Grant - “Building a Diverse Generation of Rubin Scientists”

April 11-14th 2022


## Credit: 

This notebook is based on several tutorials from DP0 and the stack club (copy and paste with some modification). We make use of the publicly available images from DECam, specifically the data set from Saha et al. (2019) with the observing program ID 2013A-0719. We further heavily collaborate with the Crowded Field Task Force, Commissioning Task Force and other Kickstarter Grants. The data reduction pipeline used was developed by the Rubin Observatory.

## Learning Objectives:
- Appreciate the notebook environment of Rubin Observatory's data access.
- Explore the data using the Butler interface.
- Differenciate between different intermediate and final data products.
- Know how to display images.
- Understand and modify standard configuration pipeline parameters and rerun the image analysis.

## Data products:

Compared to DP0 we will use the LINCC computing system to access the data. Furthermore, our data is not simulated, I ingested and reduced observed DECam images. The images we will be using are rather crowded with a focus on the Milky Way bulge. Detectors 2, 31 and 61 were ignored during the data reduction process.

### DECam
The Dark Energy Camera (DECam) is a wide-field imager mounted the on the 4m Blanco telescope. It has an 2.2 degree field with a pixelscale of 0.27"/pixel. The imaging system consits of 62 2k$\times$4k CCDs. It is therefore an ideal instrument to conduct pre-cursor studies  

## Requisites:
- Basic understanding of Python
- Github account
- Access to the LINCC cloud computing facility




In [ ]:
import os
from astropy import units as u
from astropy.coordinates import SkyCoord
import matplotlib.pylab as plt

import warnings                      # imports the warnings library
import gc                            # imports python's garbage collector
from astropy.wcs import WCS          # imports astropy's World Coordinate System function WCS

from astropy.visualization import make_lupton_rgb
import numpy as np

# Import LSST Science Pipelines packages (see pipelines.lsst.io)
import lsst.daf.base as dafBase
from lsst.daf.butler import Butler
import lsst.afw.image as afwImage
import lsst.afw.display as afwDisplay
import lsst.afw.table as afwTable
import lsst.geom as geom
import lsst.sphgeom as sphgeom

# Import Pipeline tasks 
from lsst.pipe.tasks.characterizeImage import CharacterizeImageTask
from lsst.pipe.tasks.calibrate import CalibrateTask
from lsst.meas.algorithms.detection import SourceDetectionTask
from lsst.meas.deblender import SourceDeblendTask
from lsst.meas.base import SingleFrameMeasurementTask

In [ ]:
# Claimed to be color-blind friendly

CB_color = ['#377eb8', '#ff7f00', '#4daf4a',
                  '#f781bf', '#a65628', '#984ea3',
                  '#999999', '#e41a1c', '#dede00']

In [ ]:
# Set up some plotting defaults:

params = {'axes.labelsize': 12,
          'font.size': 12,
          'legend.fontsize': 14,
          'xtick.major.width': 3,
          'xtick.minor.width': 2,
          'xtick.major.size': 12,
          'xtick.minor.size': 6,
          'xtick.direction': 'in',
          'xtick.top': True,
          'lines.linewidth': 3,
          'axes.linewidth': 3,
          'axes.labelweight': 3,
          'axes.titleweight': 3,
          'ytick.major.width': 3,
          'ytick.minor.width': 2,
          'ytick.major.size': 12,
          'ytick.minor.size': 6,
          'ytick.direction': 'in',
          'ytick.right': True,
          'figure.figsize': [8, 8],
          'figure.facecolor': 'White'
          }

plt.rcParams.update(params)

In [ ]:
# prevent some helpful but ancillary warning messages from printing
#   during some LSST DM Release calls
warnings.simplefilter("ignore", category=UserWarning)

In [ ]:
# Use lsst.afw.display with the matplotlib backend
afwDisplay.setDefaultBackend('matplotlib')

<br><br><br>

## Butler system

The amount of the large data makes is necesary to store everyting in a database. The butler is a high level access through python to the data. Data is stored in repositories. I created the DECam repository to ingest, analyise and access the DECam data. Therefore, we have to create a butler instance pointing to the directory of the repository.

In [ ]:
#We set the path of the butler repository
REPO_DIR = '/home/markusrabus/extra_disk/markusrabus/DATA/Repo_DECam'


In [ ]:
#Create the butler instance
butler = Butler(REPO_DIR)

In [ ]:
#The data inside the repository is further structured in different collections of data of the same type and propose.
#In this cell we query the butler to show us all available collections in the repositor

registry = butler.registry
for col in registry.queryCollections():
	print(col)

In [ ]:
for c in sorted(registry.queryCollections('refcats', flattenChains=True)):
    print(c, registry.getCollectionType(c))

In [ ]:
#we can also create a butler instance considering only a certain collection
butler_collection = Butler(REPO_DIR, collections='processCcdOutputs1/processed_bulge_Mar2022')

In [ ]:
for ref in butler_collection.registry.queryDatasetTypes():
    print(ref.name)

<br><br>
Some important Dataset types:
- `src`: a catalog of sources
- `calexp`: single CCD of a processed visit
<br><br>

In [ ]:
# Next we can query all available raw images in a certain filter and detector.
number_images = 0
for ref in butler.registry.queryDatasets('raw', collections='DECam/raw/all', instrument='DECam', detector=1, band='i'):
    print(ref.dataId.full)
    number_images += 1
    
print( "Number of visits: {}".format(number_images) )

In [ ]:
registry.getCollectionDocumentation('processCcdOutputs1/processed_bulge_Mar2022')

In [ ]:
datasetRefs = registry.queryDatasets(datasetType='calexp', collections='processCcdOutputs1/processed_bulge_Mar2022')

for i, ref in enumerate(datasetRefs):
    print(ref.dataId.full)
    print(ref.dataId)
    print(ref.dataId['band'])
    print(' ')


<br><br><br>

## Display images
We can display different images:
- Raw images
- Images with the instrumental signature removed (ISR)
- Calibrated exposures (calexp), which include source detection and deblending.




In [ ]:
#We start by displaying a raw images.
rawexp_g = butler.get('raw', band='g', detector=50, collections='DECam/raw/all', instrument='DECam')


In [ ]:
calexp_g = butler.get('calexp', visit=427658, band='g', detector=50, collections='processCcdOutputs1/processed_bulge_Mar2022', instrument='DECam')
#calexp_r = butler.get('calexp', visit=427655, band='r', detector=50, collections=collection, instrument='DECam')
#calexp_i = butler.get('calexp', visit=427656, band='i', detector=50, collections=collection, instrument='DECam')

In [ ]:
fig, ax = plt.subplots(1, 1, figsize=(14, 7))
display1 = afwDisplay.Display(frame=fig)
display1.scale('linear', 'zscale')
display1.mtv(calexp_g)

In [ ]:
fig, ax = plt.subplots(1, 1, figsize=(14, 7))
plt.sca(ax[0])  # set the first axis as current
display1 = afwDisplay.Display(frame=fig)
display1.scale('linear', 'zscale')
display1.mtv(calexp_g)
plt.sca(ax[1])  # set the second axis as current
display2 = afwDisplay.Display(frame=fig)
display2.scale('linear', 'zscale')
display2.mtv(calexp_r)
plt.sca(ax[2])  # set the second axis as current
display3 = afwDisplay.Display(frame=fig)
display3.scale('linear', 'zscale')
display3.mtv(calexp_i)
plt.tight_layout()
plt.show()


In [ ]:
# Print the colors associated to each plane in the mask
print("Mask plane bit definitions:\n", display1.getMaskPlaneColor())
print("\nMask plane methods:\n")


In [ ]:
# Define the pixel coordinates of a known low surface brightness "galaxy"
x_target, y_target = 800, 3700
width, height = 500, 500

#x_target, y_target = 1700, 2100
#width, height = 400, 400

xmin, ymin = x_target-width//2, y_target-height//2
point = geom.Point2D(x_target,y_target)

# Define a small region for a cutout
bbox = geom.Box2I()
bbox.include(geom.Point2I(xmin, ymin))
bbox.include(geom.Point2I(xmin + width, ymin + height))

# An alternative way to defined the same cutout region
# bbox = geom.Box2I(geom.Point2I(xmin, ymin), geom.Extent2I(width, height))

# Generate the cutout image
cutout = calexp_r.Factory(calexp_r, bbox, origin=afwImage.LOCAL, deep=False)

In [ ]:
fig, ax = plt.subplots(1, 1, figsize=(14, 7))
plt.sca(ax)  # set the first axis as current
display1 = afwDisplay.Display(frame=fig)
display1.scale('asinh', 'zscale')
display1.mtv(cutout)

In [ ]:
fig = plt.figure()
# Set the figure's projection to be the WCS of the calexp
plt.subplot(projection=WCS(calexp_r.getWcs().getFitsMetadata()))
# Display the calexp image data array using the gray colormap (cmap)
#  and use approximately the same min and max scale values as above
im = plt.imshow(calexp_r.image.array, cmap='gray', vmin=-200.0, vmax=400, origin='lower')
# Add solid white grid lines
plt.grid(color='white', ls='solid')
# Label axes
plt.xlabel('Right Ascension')
plt.ylabel('Declination')
plt.show()


In [ ]:
schema = afwTable.SourceTable.makeMinimalSchema()
print(schema)

# Create a container which will be used to record metadata
#  about algorithm execution
algMetadata = dafBase.PropertyList()
print('algMetadata: ')
algMetadata

In [ ]:
# Characterize the image properties
config = CharacterizeImageTask.ConfigClass()
config.psfIterations = 1
charImageTask = CharacterizeImageTask(config=config)

# Detect sources
config = SourceDetectionTask.ConfigClass()
# detection threshold in units of thresholdType
config.thresholdValue = 10
# units for thresholdValue
config.thresholdType = "stdev"
sourceDetectionTask = SourceDetectionTask(schema=schema, config=config)

# Deblend sources
sourceDeblendTask = SourceDeblendTask(schema=schema)

# Measure source properties
config = SingleFrameMeasurementTask.ConfigClass()
sourceMeasurementTask = SingleFrameMeasurementTask(schema=schema,
                                                   config=config,
                                                   algMetadata=algMetadata)

In [ ]:
tab = afwTable.SourceTable.make(schema)

In [ ]:
# Image characterization (this cell may take a few seconds)
result = charImageTask.run(calexp_r)

# Get the PSF at our point of interest
psf = calexp_r.getPsf()
sigma = psf.computeShape(point).getDeterminantRadius()
pixelScale = calexp_r.getWcs().getPixelScale().asArcseconds()

# The factor of 2.355 converts from std to fwhm
print('psf fwhm = {:.2f} arcsec'.format(sigma*pixelScale*2.355))

In [ ]:
# Source detection (this cell may take a few seconds)
result = sourceDetectionTask.run(tab, calexp_r)
type(result)

In [ ]:
for k, v in result.getDict().items():
    print(k, type(v))

In [ ]:
result.numPosPeaks

In [ ]:
sources = result.sources

In [ ]:
# Source deblending
sourceDeblendTask.run(calexp_r, sources)

# Source measurement (catch future warning about machine precision)
sourceMeasurementTask.run(measCat=sources, exposure=calexp_r)

In [ ]:
# The copy makes sure that the sources are sequential in memory
sources = sources.copy(True)

# Investigate the output source catalog
sources.asAstropy()

In [ ]:
# Display the cutout and sources with afw display
image = cutout.image

plt.figure()
afw_display = afwDisplay.Display()
afw_display.scale('asinh', 'zscale')
afw_display.mtv(image)
plt.gca().axis('off')

# We use display buffering to avoid re-drawing the image
#  after each source is plotted
with afw_display.Buffering():
    for s in sources:
        afw_display.dot('+', s.getX(), s.getY(), ctype=afwDisplay.RED)
        afw_display.dot('o', s.getX(), s.getY(), size=5, ctype='orange')

In [ ]:
schema2 = afwTable.SourceTable.makeMinimalSchema()

# Create a container which will be used to record metadata
#  about algorithm execution
algMetadata = dafBase.PropertyList()


# Characterize the image properties
config = CharacterizeImageTask.ConfigClass()
config.psfIterations = 1
charImageTask2 = CharacterizeImageTask(config=config)

# Detect sources
config = SourceDetectionTask.ConfigClass()
# detection threshold in units of thresholdType
config.thresholdValue = 5
# units for thresholdValue
config.thresholdType = "stdev"
sourceDetectionTask2 = SourceDetectionTask(schema=schema, config=config)

# Deblend sources
sourceDeblendTask2 = SourceDeblendTask(schema=schema2)

# Measure source properties
config = SingleFrameMeasurementTask.ConfigClass()
sourceMeasurementTask = SingleFrameMeasurementTask(schema=schema2,
                                                   config=config,
                                                   algMetadata=algMetadata)

In [ ]:
tab = afwTable.SourceTable.make(schema2)

In [ ]:
result2 = charImageTask.run(calexp_r)
result2 = sourceDetectionTask.run(tab, calexp_r)
sources2 = result2.sources
sourceDeblendTask.run(calexp_r, sources2)
sourceMeasurementTask.run(measCat=sources2, exposure=calexp_r)

In [ ]:
# The copy makes sure that the sources are sequential in memory
sources2 = sources2.copy(True)



In [ ]:
# Display the cutout and sources with afw display
image = cutout.image

plt.figure()
afw_display = afwDisplay.Display()
afw_display.scale('asinh', 'zscale')
afw_display.mtv(image)
plt.gca().axis('off')


        
        
with afw_display.Buffering():
    for s in sources:
        afw_display.dot('+', s.getX(), s.getY(), ctype=afwDisplay.RED, size=5)
        afw_display.dot('o', s.getX(), s.getY(), size=7, ctype='green')
        
        
# We use display buffering to avoid re-drawing the image
#  after each source is plotted
with afw_display.Buffering():
    for s in sources2:
        afw_display.dot('+', s.getX(), s.getY(), ctype=afwDisplay.BLUE)
        afw_display.dot('o', s.getX(), s.getY(), size=5, ctype='orange')

In [ ]:
sources_r1 = butler.get('src', visit=427335, band='r', detector=50, collections=collection, instrument='DECam')
sources_r2 = butler.get('src', visit=427350, band='r', detector=50, collections=collection, instrument='DECam')

In [ ]:
sources_r1

In [ ]:
sources_r2